In [1]:
import pandas as pd
import numpy as np
import nltk
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem.porter import *

In [2]:
df=pd.read_csv('got_scripts_breakdown.csv', sep=';')
df.head()

,Column1,Season,Episode,Sentence,Name,N_serie,N_Season,Emision Date
0,1,Season 1,winter is coming,What d’you expect? They’re savages One lot ste...,waymar royce,1,1,17/04/2011
1,2,Season 1,winter is coming,I’ve never seen wildlings do a thing like this...,will,1,1,17/04/2011
2,3,Season 1,winter is coming,How close did you get?,waymar royce,1,1,17/04/2011
3,4,Season 1,winter is coming,Close as any man would,will,1,1,17/04/2011
4,5,Season 1,winter is coming,We should head back to the wall,gared,1,1,17/04/2011


In [3]:
lines = df[["Sentence"]]
lines.reset_index(level=0, inplace=True)
documents = lines
#lines = ' '.join(lines)

print(len(documents))
print(documents[:5])

22510
   index                                           Sentence
0      0  What d’you expect? They’re savages One lot ste...
1      1  I’ve never seen wildlings do a thing like this...
2      2                             How close did you get?
3      3                             Close as any man would
4      4                    We should head back to the wall


In [4]:
def lemmatize_stemming(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

In [5]:
doc_sample = documents[documents['index'] == 4314].values[0][1]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))
    

original document: 
['Come', 'back', 'here!', 'Help', 'us,', 'boy!', 'A', 'man', 'can', 'fight!', 'Free', 'us']


 tokenized and lemmatized document: 
['come', 'help', 'boy', 'man', 'fight', 'free']


In [6]:
processed_docs = documents['Sentence'].map(preprocess)
processed_docs[:10]

0    [expect, savag, lot, steal, goat, lot, know, r...
1    [ve, see, wildl, thing, like, ve, see, thing, ...
2                                              [close]
3                                         [close, man]
4                                         [head, wall]
5                                     [dead, frighten]
6            [order, track, wildl, track, win, troubl]
7                          [think, ll, ask, die, hors]
8                                     [kill, children]
9    [good, thing, children, want, run, away, south...
Name: Sentence, dtype: object

In [7]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break


0 expect
1 goat
2 know
3 lot
4 piec
5 rip
6 savag
7 steal
8 life
9 like
10 see


In [28]:
# dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [8]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4314]

[(15, 1), (78, 1), (82, 1), (277, 1), (455, 1), (1903, 1)]

In [9]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.2904910605228106),
 (1, 0.35863499479757566),
 (2, 0.1436307125243663),
 (3, 0.5479844543540072),
 (4, 0.3414600707102706),
 (5, 0.3847096314396112),
 (6, 0.3433164245483797),
 (7, 0.28916516507423884)]


In [10]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [11]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.060*"know" + 0.030*"im" + 0.021*"jon" + 0.019*"come" + 0.011*"take" + 0.010*"snow" + 0.008*"mean" + 0.008*"peopl" + 0.008*"queen" + 0.007*"man"
Topic: 1 
Words: 0.033*"need" + 0.026*"ye" + 0.020*"help" + 0.017*"leav" + 0.014*"want" + 0.014*"lord" + 0.012*"thank" + 0.011*"cours" + 0.010*"men" + 0.010*"time"
Topic: 2 
Words: 0.027*"good" + 0.017*"wall" + 0.016*"go" + 0.015*"way" + 0.014*"night" + 0.014*"say" + 0.013*"walk" + 0.012*"believ" + 0.012*"get" + 0.012*"dead"
Topic: 3 
Words: 0.021*"like" + 0.019*"look" + 0.015*"let" + 0.013*"hear" + 0.013*"want" + 0.011*"queen" + 0.011*"arya" + 0.010*"stop" + 0.010*"die" + 0.010*"think"
Topic: 4 
Words: 0.024*"tell" + 0.024*"north" + 0.018*"king" + 0.016*"stark" + 0.011*"kill" + 0.011*"know" + 0.011*"he" + 0.010*"lord" + 0.010*"serv" + 0.009*"war"
Topic: 5 
Words: 0.033*"king" + 0.018*"land" + 0.014*"love" + 0.014*"girl" + 0.013*"brother" + 0.011*"saw" + 0.011*"sister" + 0.010*"boy" + 0.009*"rememb" + 0.008*"that"
Topic: 6 
W

In [12]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWord: {}'.format(idx, topic))

Topic: 0 
Word: 0.022*"come" + 0.011*"fight" + 0.009*"time" + 0.009*"go" + 0.009*"father" + 0.009*"love" + 0.007*"want" + 0.007*"know" + 0.007*"ve" + 0.007*"your"
Topic: 1 
Word: 0.013*"like" + 0.009*"girl" + 0.009*"happen" + 0.008*"lord" + 0.008*"command" + 0.008*"ask" + 0.008*"need" + 0.007*"ll" + 0.006*"your" + 0.006*"right"
Topic: 2 
Word: 0.016*"king" + 0.013*"die" + 0.011*"mother" + 0.009*"im" + 0.008*"dont" + 0.008*"north" + 0.008*"believ" + 0.008*"take" + 0.008*"dead" + 0.007*"sam"
Topic: 3 
Word: 0.022*"dont" + 0.018*"want" + 0.015*"good" + 0.013*"hear" + 0.013*"right" + 0.010*"man" + 0.009*"father" + 0.009*"say" + 0.008*"come" + 0.008*"that"
Topic: 4 
Word: 0.032*"know" + 0.013*"kill" + 0.010*"ll" + 0.009*"lord" + 0.006*"lannist" + 0.006*"speak" + 0.006*"tell" + 0.006*"saw" + 0.006*"winterfel" + 0.006*"head"
Topic: 5 
Word: 0.015*"leav" + 0.013*"help" + 0.009*"talk" + 0.009*"stark" + 0.008*"your" + 0.007*"home" + 0.007*"want" + 0.007*"wait" + 0.007*"know" + 0.006*"dont"
Topic

In [13]:
processed_docs[4314]

['come', 'help', 'boy', 'man', 'fight', 'free']

In [14]:
for index, score in sorted(lda_model[bow_corpus[4314]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8713999390602112	 
Topic: 0.033*"need" + 0.026*"ye" + 0.020*"help" + 0.017*"leav" + 0.014*"want" + 0.014*"lord" + 0.012*"thank" + 0.011*"cours" + 0.010*"men" + 0.010*"time"

Score: 0.01429202500730753	 
Topic: 0.032*"come" + 0.028*"men" + 0.014*"hand" + 0.012*"your" + 0.011*"king" + 0.009*"hold" + 0.009*"fight" + 0.008*"protect" + 0.008*"want" + 0.008*"end"

Score: 0.014289546757936478	 
Topic: 0.033*"king" + 0.018*"land" + 0.014*"love" + 0.014*"girl" + 0.013*"brother" + 0.011*"saw" + 0.011*"sister" + 0.010*"boy" + 0.009*"rememb" + 0.008*"that"

Score: 0.01428940985351801	 
Topic: 0.060*"know" + 0.030*"im" + 0.021*"jon" + 0.019*"come" + 0.011*"take" + 0.010*"snow" + 0.008*"mean" + 0.008*"peopl" + 0.008*"queen" + 0.007*"man"

Score: 0.014288964681327343	 
Topic: 0.024*"tell" + 0.024*"north" + 0.018*"king" + 0.016*"stark" + 0.011*"kill" + 0.011*"know" + 0.011*"he" + 0.010*"lord" + 0.010*"serv" + 0.009*"war"

Score: 0.014288806356489658	 
Topic: 0.027*"good" + 0.017*"wall" + 0.0

In [15]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4314]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8713904023170471	 
Topic: 0.022*"come" + 0.011*"fight" + 0.009*"time" + 0.009*"go" + 0.009*"father" + 0.009*"love" + 0.007*"want" + 0.007*"know" + 0.007*"ve" + 0.007*"your"

Score: 0.014298840425908566	 
Topic: 0.015*"leav" + 0.013*"help" + 0.009*"talk" + 0.009*"stark" + 0.008*"your" + 0.007*"home" + 0.007*"want" + 0.007*"wait" + 0.007*"know" + 0.006*"dont"

Score: 0.014290248043835163	 
Topic: 0.030*"ye" + 0.020*"lord" + 0.015*"tell" + 0.014*"oh" + 0.011*"hodor" + 0.010*"thank" + 0.009*"jon" + 0.007*"snow" + 0.006*"sansa" + 0.006*"boy"

Score: 0.014290128834545612	 
Topic: 0.022*"dont" + 0.018*"want" + 0.015*"good" + 0.013*"hear" + 0.013*"right" + 0.010*"man" + 0.009*"father" + 0.009*"say" + 0.008*"come" + 0.008*"that"

Score: 0.01428909134119749	 
Topic: 0.013*"like" + 0.009*"girl" + 0.009*"happen" + 0.008*"lord" + 0.008*"command" + 0.008*"ask" + 0.008*"need" + 0.007*"ll" + 0.006*"your" + 0.006*"right"

Score: 0.014289065264165401	 
Topic: 0.016*"king" + 0.013*"die" + 0.011